# Qdrant Rust Client Example

## 1. 크레잇 설치

In [4]:
:dep qdrant-client = {version = "1.15.0", features = ["serde"]}
:dep tokio         = {version = "1.47.1", features = ["full"]}
:dep dotenv        = "0.15.0"

## 2. 모듈 임포트

In [5]:
use qdrant_client::{Qdrant, QdrantError};

## 3. Qdrnt Rust Client 생성

In [7]:
let client = Qdrant::from_url("http://localhost:6334").build()?;

## 4. 컬렉션 목록 확인

In [8]:
let collections_list = client.list_collections().await?;
println!("기존 컬렉션: {:?}", collections_list);

기존 컬렉션: ListCollectionsResponse { collections: [], time: 6.849e-6 }


## 5. 신규 컬렉션 (있다면 삭제 후) 생성

In [27]:
use qdrant_client::qdrant::{CreateCollectionBuilder, Distance, VectorParamsBuilder};

let collection_name = "my_collection";
let vector_size = 100;            // 100차원 벡터
let distance = Distance::Cosine;  // 코사인 유사도

let exists = client.collection_exists(collection_name).await?;
println!("컬렉션 '{}' 존재 여부: {}", collection_name, exists);

match exists {
    true => {
        println!("컬렉션 '{}'이 이미 존재합니다.", collection_name);
    },
    false => {
        client.create_collection(CreateCollectionBuilder::new(collection_name).vectors_config(VectorParamsBuilder::new(vector_size, distance))).await?;
        println!("컬렉션 '{}'을 생성했습니다.", collection_name);
    }
}

컬렉션 'my_collection' 존재 여부: true
컬렉션 'my_collection'이 이미 존재합니다.


()

## 6. 포인트 추가 (ID, 벡터, 페이로드)

In [17]:
:dep serde_json = "1.0.145"

use qdrant_client::qdrant::{PointStruct, UpsertPointsBuilder};
use qdrant_client::{Payload};

// 샘플 페이로드 생성
let payload1: Payload = serde_json::json!({"name": "문서1", "category": "기술", "score": 95}).try_into().unwrap();
let payload2: Payload = serde_json::json!({"name": "문서2", "category": "과학", "score": 87}).try_into().unwrap();
let payload3: Payload = serde_json::json!({"name": "문서3", "category": "기술", "score": 92}).try_into().unwrap();

// 포인트 생성 (ID, 벡터, 페이로드)
let points = vec![
    PointStruct::new(1, vec![0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], payload1),
    PointStruct::new(2, vec![1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1], payload2),
    PointStruct::new(3, vec![0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], payload3),
];

// 포인트 삽입
client.upsert_points(UpsertPointsBuilder::new(collection_name, points)).await?;

println!("3개의 포인트가 성공적으로 삽입되었습니다!");

3개의 포인트가 성공적으로 삽입되었습니다!


## 7. 벡터 검색

In [28]:
use qdrant_client::qdrant::{SearchParamsBuilder, SearchPointsBuilder};
use std::collections::HashMap;

// 쿼리 벡터로 검색
let query_vector = vec![0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.1];

// 상위 5개 결과
// 페이로드 포함
let search_result = client.search_points(SearchPointsBuilder::new(collection_name, query_vector, 5).with_payload(true).params(SearchParamsBuilder::default().hnsw_ef(128).exact(false))).await?;
println!("검색 결과:");

for point in search_result.result {
    // ID는 버전에 따라 Option<PointId> 또는 PointId일 수 있으므로 Debug로 안전하게 출력
    let id_str = format!("{:?}", point.id);
    println!("ID: {}, Score: {:.4}", id_str, point.score);
    // payload는 HashMap<String, Value>인 버전이 있으므로 빈 맵이 아닐 때만 출력
    if !point.payload.is_empty() {
        println!("  메타데이터: {:?}", point.payload);
    }
    println!();
}

검색 결과:
ID: Some(PointId { point_id_options: Some(Num(3)) }), Score: 0.8864
  메타데이터: {"score": Value { kind: Some(IntegerValue(92)) }, "name": Value { kind: Some(StringValue("문서3")) }, "category": Value { kind: Some(StringValue("기술")) }}

ID: Some(PointId { point_id_options: Some(Num(1)) }), Score: 0.8831
  메타데이터: {"name": Value { kind: Some(StringValue("문서1")) }, "score": Value { kind: Some(IntegerValue(95)) }, "category": Value { kind: Some(StringValue("기술")) }}

ID: Some(PointId { point_id_options: Some(Num(2)) }), Score: 0.6883
  메타데이터: {"name": Value { kind: Some(StringValue("문서2")) }, "category": Value { kind: Some(StringValue("과학")) }, "score": Value { kind: Some(IntegerValue(87)) }}



()

## 8. 필터 검색

In [34]:
use qdrant_client::qdrant::{Condition, Filter};

let filtered_search = client.search_points(SearchPointsBuilder::new(collection_name, vec![0.5; 10], 10).filter(Filter::all([Condition::matches("category", "기술".to_string())])).with_payload(true)).await?;

println!("필터링된 검색 결과 (카테고리='기술'):");
for point in filtered_search.result {
    println!("ID: {:?}, Score: {:.4}", point.id.unwrap(), point.score);
    if !point.payload.is_empty() {
        println!("  이름: {:?}", point.payload.get("name").unwrap().as_str().unwrap());
        println!("  카테고리: {:?}", point.payload.get("category").unwrap().as_str().unwrap());
        println!("  점수: {:?}", point.payload.get("score").unwrap().as_integer().unwrap());
    }
    println!();
}

필터링된 검색 결과 (카테고리='기술'):
ID: PointId { point_id_options: Some(Num(3)) }, Score: 1.0000
  이름: "문서3"
  카테고리: "기술"
  점수: 92

ID: PointId { point_id_options: Some(Num(1)) }, Score: 0.8864
  이름: "문서1"
  카테고리: "기술"
  점수: 95



()

## 9. 포인트 정보 조회

In [39]:
use qdrant_client::qdrant::{GetPointsBuilder};

let point_info = client.get_points(GetPointsBuilder::new(collection_name, vec![0.into(), 30.into(), 100.into()])).await?;
    println!("포인트 1 정보: {:?}", point_info);

포인트 1 정보: GetResponse { result: [], time: 0.000529895, usage: None }


## 10. 컬렉션 통계 확인

In [40]:
let collection_info = client.collection_info(collection_name).await?;
println!("컬렉션 정보: {:?}", collection_info);

컬렉션 정보: GetCollectionInfoResponse { result: Some(CollectionInfo { status: Green, optimizer_status: Some(OptimizerStatus { ok: true, error: "" }), vectors_count: None, segments_count: 8, config: Some(CollectionConfig { params: Some(CollectionParams { shard_number: 1, on_disk_payload: true, vectors_config: Some(VectorsConfig { config: Some(Params(VectorParams { size: 10, distance: Cosine, hnsw_config: None, quantization_config: None, on_disk: None, datatype: None, multivector_config: None })) }), replication_factor: Some(1), write_consistency_factor: Some(1), read_fan_out_factor: None, sharding_method: None, sparse_vectors_config: None }), hnsw_config: Some(HnswConfigDiff { m: Some(16), ef_construct: Some(100), full_scan_threshold: Some(10000), max_indexing_threads: Some(0), on_disk: Some(false), payload_m: None }), optimizer_config: Some(OptimizersConfigDiff { deleted_threshold: Some(0.2), vacuum_min_vector_number: Some(1000), default_segment_number: Some(0), max_segment_size: None, mem